<a href="https://colab.research.google.com/github/PrashanthPrince/Sofa/blob/main/sofa_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

""" 

Author : Prashanth Prince

Date : 08 Dec 2020

Note: The conditions for calculating the sofa score are defined according to the details given in the following Wikipedia page:
[Sofa Score](https://https://en.wikipedia.org/wiki/SOFA_score)



"""

In [71]:
import psycopg2
import pandas as pd

The following code cell is used to connect to the PostreSQL and print out the connection details.

In [72]:
connection = psycopg2.connect(user = "datascientist",
                                  password = "candidate",
                                  host = "3.7.155.14",
                                  port = "5432",
                                  database = "mimic")

cursor = connection.cursor()
# Print PostgreSQL Connection properties
print ( connection.get_dsn_parameters(),"\n")

{'user': 'datascientist', 'dbname': 'mimic', 'host': '3.7.155.14', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



The following code cell is used to retrieve the cardiovascular system values from the database and store it in a pandas dataframe.

In [74]:
get_cardiovascular_system_values_query = '''select e.icustay_id, e.vasonum epinephrine_value, d.vasonum dopamine_value, n.vasonum _value  from mimiciii.epinephrinedurations e inner join mimiciii.dopaminedurations d on e.icustay_id=d.icustay_id inner join mimiciii.norepinephrinedurations n on d.icustay_id=n.icustay_id order by e.icustay_id limit 500;'''
cursor.execute(get_cardiovascular_system_values_query)
cardiovascular_system_values = cursor.fetchall()
df_cardiovascular_system_values = pd.DataFrame(cardiovascular_system_values, columns=['icustay_id', 'epinephrine_value', 'dopamine_value', 'norepinephrine_value'])
#df_cardiovascular_system_values.head()

The following code cell is used to retrieve the nervous system values from the database and store it in a pandas dataframe.

In [78]:
get_nervous_system_values_query = '''SELECT subject_id, hadm_id, valuenum FROM mimiciii.labevents WHERE valueuom='%' ORDER BY subject_id  LIMIT 500;'''
cursor.execute(get_nervous_system_values_query)
nervous_system_values = cursor.fetchall()

In [79]:
df_nervous_system_values = pd.DataFrame(nervous_system_values, columns=['subject_id', 'hadm_id', 'glasgow_coma_scale'])
#df_nervous_system_values.head()

The following code cell is used to retrieve the respiratory system values from the database and store it in a pandas dataframe.

In [82]:
get_respiratory_system_values_query = '''SELECT subject_id, hadm_id, icustay_id, valuenum FROM mimiciii.chartevents WHERE valueuom='mmHg' ORDER BY subject_id  LIMIT 500;'''
cursor.execute(get_cardiovascular_system_values_query)
respiratory_system_values = cursor.fetchall()
df_respiratory_system_values = pd.DataFrame(respiratory_system_values, columns=['subject_id', 'hadm_id', 'icustay_id', 'pao2'])
#df_respiratory_system_values.head()

The following code cell is used to retrieve the liver, kidneys and coagulation values from the database and store it in a pandas dataframe. These are measured in bilirubin, creatinine and platelets values respectively.

In [84]:
get_lkc_system_values_query = '''SELECT subject_id, hadm_id, icustay_id, (bilirubin_min+bilirubin_max)/2.0 as bilirubin, (creatinine_min+creatinine_max)/2.0 as creatinine, (platelet_min+platelet_max)/2.0 as platelet  FROM mimiciii.labsfirstday ORDER BY subject_id  LIMIT 500;'''
cursor.execute(get_lkc_system_values_query)
lkc_system_values = cursor.fetchall()

In [85]:
df_lkc_system_values = pd.DataFrame(lkc_system_values, columns=['subject_id', 'hadm_id', 'icustay_id', 'bilirubin', 'creatinine', 'platelet'])
#df_lkc_system_values.head()

Joining lkc and nervous systems 

In [86]:
df_lkc_nervous = pd.merge(df_lkc_system_values, df_nervous_system_values, on='subject_id')

#df_lkc_nervous

Joining previous dataframe with cardiovascular system values.

In [88]:
df_lkc_nervous_cardio = pd.merge(df_lkc_nervous, df_cardiovascular_system_values, on='icustay_id', how='outer')

#df_lkc_nervous_cardio

In [91]:
df_lkc_nervous_cardio.isna().sum()

subject_id              500
hadm_id_x               500
icustay_id                0
bilirubin               538
creatinine              538
platelet                500
hadm_id_y               839
glasgow_coma_scale      500
epinephrine_value       500
dopamine_value          500
norepinephrine_value    500
dtype: int64

Handling Missing Values. Filling forward and Backward since there are repeated entries.

In [92]:
df_lkc_nervous_cardio['epinephrine_value'].fillna(method='bfill',inplace=True)
df_lkc_nervous_cardio['dopamine_value'].fillna(method='bfill',inplace=True)
df_lkc_nervous_cardio['norepinephrine_value'].fillna(method='bfill',inplace=True)

df_lkc_nervous_cardio['bilirubin'].fillna(method='bfill',inplace=True)
df_lkc_nervous_cardio['creatinine'].fillna(method='bfill',inplace=True)

df_lkc_nervous_cardio['bilirubin'].fillna(method='ffill',inplace=True)
df_lkc_nervous_cardio['creatinine'].fillna(method='ffill',inplace=True)

df_lkc_nervous_cardio['subject_id'].fillna(method='ffill',inplace=True)

df_lkc_nervous_cardio['platelet'].fillna(method='ffill',inplace=True)

df_lkc_nervous_cardio['glasgow_coma_scale'].fillna(method='ffill',inplace=True)

In [95]:
df_lkc_nervous_cardio['subject_id'].isna().sum()

0

In [96]:
df_lkc_nervous_cardio['icustay_id'].isna().sum()

0

In [97]:
connection.close()
cursor.close()

In [111]:
#df_lkc_nervous_cardio

Defining Functions for each health system and assigning sofa values according to the measured quantity.

Note: The conditions are defined according to the details given in the following Wikipedia page:
[Sofa Score](https://https://en.wikipedia.org/wiki/SOFA_score)

In [99]:
def kidney(row):
  if row['creatinine']<1.2:
    val = 0
  elif row['creatinine']>=1.2 and row['creatinine']<=1.9:
    val = 1
  elif row['creatinine']>=2.0 and row['creatinine']<=3.4:
    val = 2
  elif row['creatinine']>=3.5 and row['creatinine']<=4.9:
    val = 3
  else:
    val = 4
  return val

def coagulation(row):
  if row['platelet']>=150:
    val = 0
  elif row['platelet']<150:
    val = 1
  elif row['platelet']<100:
    val = 2
  elif row['platelet']<50:
    val = 3
  else:
    val = 4
  return val

def liver(row):
  if row['bilirubin']<1.2:
    val = 0
  elif row['bilirubin']>=1.2 and row['bilirubin']<=1.9:
    val = 1
  elif row['bilirubin']>=2.0 and row['bilirubin']<=5.9:
    val = 2
  elif row['bilirubin']>=6 and row['bilirubin']<=11.9:
    val = 3
  else:
    val = 4
  return val

def cardiovascular(row):
  if row['dopamine_value']<=5:
    val = 2
  elif row['dopamine_value']>5 or row['epinephrine_value']<=0.1 or row['norepinephrine_value']<=0.1:
    val = 3
  elif row['dopamine_value']>15 or row['epinephrine_value']>0.1 or row['norepinephrine_value']>0.1:
    val = 4
  return val

def nervous(row):
  if row['glasgow_coma_scale']==15:
    val = 0
  elif row['glasgow_coma_scale']>=13 and row['glasgow_coma_scale']<=14:
    val = 1
  elif row['glasgow_coma_scale']>=10 and row['glasgow_coma_scale']<=12:
    val = 2
  elif row['glasgow_coma_scale']>=6 and row['glasgow_coma_scale']<=9:
    val = 3
  else:
    val = 4
  return val


In [100]:
def sofa(row):
  val = kidney(row)
  val += coagulation(row)
  val += liver(row)
  val += cardiovascular(row)
  val += nervous(row)
  return val

In [101]:
df_lkc_nervous_cardio['sofa'] = df_lkc_nervous_cardio.apply(sofa,axis=1)

In [108]:
#df_lkc_nervous_cardio

Sofa score is calculated for the first 1000 patients sorted according to their subject_id. The same method / algorithm mentioned in this code can be used to calculate the sofa score of all the patients. 

In [110]:
df_lkc_nervous_cardio['sofa']

0      8
1      8
2      8
3      8
4      8
      ..
995    7
996    7
997    7
998    7
999    7
Name: sofa, Length: 1000, dtype: int64